<a href="https://colab.research.google.com/github/nolanwelch/ai-factcheck/blob/main/proof-of-concept/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install torch python-dotenv transformers openai pydantic

In [3]:
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  from google.colab import userdata
  hf_token_present = userdata.get("HF_TOKEN") is not None
else:
  import os
  import dotenv
  dotenv.load_dotenv()
  hf_token_present = "HF_TOKEN" in os.environ

assert hf_token_present, "Must set the HF_TOKEN environment variable"

In [ ]:
from transformers import pipeline

NLI_MODEL = "cmarkea/bloomz-7b1-mt-nli"

nli_classifier = pipeline("zero-shot-classification", model=NLI_MODEL)

config.json:   0%|          | 0.00/980 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/31.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]